In [2]:
!pip install kneed


^C


In [3]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import Birch
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import SpectralClustering
#from kneed import KneeLocator
from sklearn.metrics import silhouette_score

In [4]:
df = pd.read_csv('mental.csv')
df.head()

,Entity,Code,Year,Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized,Depressive disorders (share of population) - Sex: Both - Age: Age-standardized,Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized,Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized,Eating disorders (share of population) - Sex: Both - Age: Age-standardized
0,Afghanistan,AFG,1990,0.223206,4.996118,4.713314,0.703023,0.127700
1,Afghanistan,AFG,1991,0.222454,4.989290,4.702100,0.702069,0.123256
2,Afghanistan,AFG,1992,0.221751,4.981346,4.683743,0.700792,0.118844
3,Afghanistan,AFG,1993,0.220987,4.976958,4.673549,0.700087,0.115089
4,Afghanistan,AFG,1994,0.220183,4.977782,4.670810,0.699898,0.111815


Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
import numpy as np
dataset_dimensions = df.shape
# Calculate the total number of elements in the dataframe
total_elements = np.product(dataset_dimensions)

# Calculate the number of elements to replace with NaN
num_elements_to_replace = int(total_elements * 0.10)

# Generate random indices to replace with NaN
np.random.seed(42) # Set seed for reproducibility
rows_to_replace = np.random.randint(0, dataset_dimensions[0], size=num_elements_to_replace)
cols_to_replace = np.random.randint(0, dataset_dimensions[1], size=num_elements_to_replace)

# Replace the selected elements with NaN
for row, col in zip(rows_to_replace, cols_to_replace):
    df.iat[row, col] = np.nan

# Verify by displaying the first few rows of the dataframe and count of NaN values
df.head(), df.isna().sum()


(        Entity Code    Year  \
 0  Afghanistan  AFG  1990.0   
 1  Afghanistan  AFG  1991.0   
 2  Afghanistan  AFG  1992.0   
 3  Afghanistan  AFG  1993.0   
 4  Afghanistan  AFG  1994.0   
 
    Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized  \
 0                                           0.223206                                   
 1                                           0.222454                                   
 2                                           0.221751                                   
 3                                           0.220987                                   
 4                                           0.220183                                   
 
    Depressive disorders (share of population) - Sex: Both - Age: Age-standardized  \
 0                                           4.996118                                
 1                                                NaN                                
 2          

In [11]:
from sklearn.impute import SimpleImputer

def clean_dataset(dataframe, drop_column_threshold=0.5, drop_row_threshold=0.5, numeric_impute_strategy='mean', categorical_impute_strategy='most_frequent'):
    """
    Cleans the provided DataFrame by imputing missing values, and optionally dropping columns and rows with a high percentage of missing values.
    
    :param dataframe: pandas DataFrame to be cleaned
    :param drop_column_threshold: Fraction threshold of missing values for dropping a column (default: 0.5)
    :param drop_row_threshold: Fraction threshold of missing values for dropping a row (default: 0.5)
    :param numeric_impute_strategy: Strategy for imputing missing values in numeric columns ('mean', 'median')
    :param categorical_impute_strategy: Strategy for imputing missing values in categorical columns ('most_frequent', 'constant')
    :return: Cleaned pandas DataFrame
    """
    df_cleaned = dataframe.copy()
    
    # Drop columns with missing value rate higher than threshold
    missing_col_fraction = df_cleaned.isnull().mean()
    columns_to_drop = missing_col_fraction[missing_col_fraction > drop_column_threshold].index
    df_cleaned.drop(columns=columns_to_drop, inplace=True)
    
    # Impute missing values for numeric columns
    numeric_columns = df_cleaned.select_dtypes(include=['number']).columns
    numeric_imputer = SimpleImputer(strategy=numeric_impute_strategy)
    df_cleaned[numeric_columns] = numeric_imputer.fit_transform(df_cleaned[numeric_columns])
    
    # Impute missing values for categorical columns
    categorical_columns = df_cleaned.select_dtypes(include=['object', 'category']).columns
    if categorical_impute_strategy == 'constant':
        categorical_imputer = SimpleImputer(strategy='constant', fill_value='missing')
    else:
        categorical_imputer = SimpleImputer(strategy=categorical_impute_strategy)
    df_cleaned[categorical_columns] = categorical_imputer.fit_transform(df_cleaned[categorical_columns])
    
    # Drop rows with missing value rate higher than threshold
    missing_row_fraction = df_cleaned.isnull().mean(axis=1)
    rows_to_drop = missing_row_fraction[missing_row_fraction > drop_row_threshold].index
    df_cleaned.drop(index=rows_to_drop, inplace=True)

    #remove unrelated columns
   
    
    return df_cleaned

# Applying the cleaning function to our dataset
df_cleaned = clean_dataset(df)

# Display changes in the cleaned dataset: first few rows and NaN counts
df_cleaned.head()


,Entity,Code,Year,Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized,Depressive disorders (share of population) - Sex: Both - Age: Age-standardized,Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized,Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized,Eating disorders (share of population) - Sex: Both - Age: Age-standardized
0,Afghanistan,AFG,1990.0,0.223206,4.996118,4.713314,0.703023,0.127700
1,Afghanistan,AFG,1991.0,0.222454,3.767303,4.100860,0.702069,0.123256
2,Afghanistan,AFG,1992.0,0.221751,4.981346,4.683743,0.700792,0.195365
3,Afghanistan,AFG,1993.0,0.220987,3.767303,4.673549,0.700087,0.115089
4,Afghanistan,AFG,1994.0,0.220183,4.977782,4.670810,0.699898,0.195365


In [5]:

data = pd.read_csv('mental_cleaned.csv',encoding='utf-8')

In [6]:
data.columns

Index(['Entity', 'Code', 'Year',
       'Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized',
       'Depressive disorders (share of population) - Sex: Both - Age: Age-standardized',
       'Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized',
       'Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized',
       'Eating disorders (share of population) - Sex: Both - Age: Age-standardized'],
      dtype='object')

In [7]:
#rename columns
data.columns = ['Entity','Code', 'Year', 'Schizphrenia', 'Depressive_disorders','Anxiety_disorders','Bipolar_disorder', 'Eating_disorders']
data.head()

,Entity,Code,Year,Schizphrenia,Depressive_disorders,Anxiety_disorders,Bipolar_disorder,Eating_disorders
0,Afghanistan,AFG,1990.0,0.223206,4.996118,4.713314,0.703023,0.127700
1,Afghanistan,AFG,1991.0,0.222454,3.767303,4.100860,0.702069,0.123256
2,Afghanistan,AFG,1992.0,0.221751,4.981346,4.683743,0.700792,0.195365
3,Afghanistan,AFG,1993.0,0.220987,3.767303,4.673549,0.700087,0.115089
4,Afghanistan,AFG,1994.0,0.220183,4.977782,4.670810,0.699898,0.195365


In [12]:
from sklearn.cluster import KMeans
import plotly.express as px
import numpy as np

# Selecting only the numerical columns related to mental disorders
features = data.iloc[:, 3:]
print(features.head())

# elbow plot
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(features)
    inertia.append(kmeans.inertia_)

# Plotting the elbow plot with plotly
fig = px.line(x=range(1, 11), y=inertia, title='Elbow Plot', labels={'x': 'Number of Clusters', 'y': 'Inertia'})
fig.show()


   Schizphrenia  Depressive_disorders  Anxiety_disorders  Bipolar_disorder  \
0      0.223206              4.996118           4.713314          0.703023   
1      0.222454              3.767303           4.100860          0.702069   
2      0.221751              4.981346           4.683743          0.700792   
3      0.220987              3.767303           4.673549          0.700087   
4      0.220183              4.977782           4.670810          0.699898   

   Eating_disorders  
0          0.127700  
1          0.123256  
2          0.195365  
3          0.115089  
4          0.195365  


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6420 entries, 0 to 6419
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Entity                6420 non-null   object 
 1   Code                  6420 non-null   object 
 2   Year                  6420 non-null   float64
 3   Schizphrenia          6420 non-null   float64
 4   Depressive_disorders  6420 non-null   float64
 5   Anxiety_disorders     6420 non-null   float64
 6   Bipolar_disorder      6420 non-null   float64
 7   Eating_disorders      6420 non-null   float64
dtypes: float64(6), object(2)
memory usage: 401.4+ KB


In [10]:
features.isna().sum()


NameError: name 'features' is not defined

In [11]:
try:
    config = get_config()
    if config:  # Check if config is not None
        config = config.split()
        if config[0] == b"OpenBLAS":
            kmeans = KMeans(n_clusters=2, random_state=42)
            kmeans.fit(features)
except:
    # Handle potential exceptions during get_config()
    pass
